## Description

In [43]:
# Arguments
REWARD = -0.01 # constant reward for non-terminal states
DISCOUNT = 0.99
MAX_ERROR = 10**(-3)

# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 3
NUM_COL = 4
U = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]


## Visualization Function

In [44]:
# Visualization
def printEnvironment(arr, policy=False):
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r == c == 1:
                val = "▉"
            elif r <= 1 and c == 3:
                val = "✅" if r == 0 else "💀"
            else:
                if policy:
                    val = ["🢃", "🢀", "🢁", "🢂"][arr[r][c]]
                else:
                    val = str(arr[r][c])
            res += " " + val[:5].ljust(7) + " |" # format
        res += "\n"
    print(res)

## Get Utility

In [45]:
# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    dr, dc = ACTIONS[action]
    newR, newC = r+dr, c+dc
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

## Value Iteration Algorithm


In [46]:
# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    u = REWARD
    u += 0.1 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 0.8 * DISCOUNT * getU(U, r, c, action)
    u += 0.1 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

def valueIteration(U):
    print("During the value iteration:\n")
    while True:
        nextU = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                nextU[r][c] = max([calculateU(U, r, c, action) for action in range(NUM_ACTIONS)]) # Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        printEnvironment(U)
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

## Get Optimal Policy

In [47]:
# Get the optimal policy from U
def getOptimalPolicy(U):
    policy = [[-1, -1, -1, -1] for i in range(NUM_ROW)]
    for r in range(NUM_ROW):
        for c in range(NUM_COL):
            if (r <= 1 and c == 3) or (r == c == 1):
                continue
            # Choose the action that maximizes the utility
            maxAction, maxU = None, -float("inf")
            for action in range(NUM_ACTIONS):
                u = calculateU(U, r, c, action)
                if u > maxU:
                    maxAction, maxU = action, u
            policy[r][c] = maxAction
    return policy

## Main

In [48]:
# Print the initial environment
print("The initial U is:\n")
printEnvironment(U)

# Value iteration
U = valueIteration(U)

# Get the optimal policy from U and print it
policy = getOptimalPolicy(U)
print("The optimal policy is:\n")
printEnvironment(policy, True)

The initial U is:

| 0       | 0       | 0       | ✅       |
| 0       | ▉       | 0       | 💀       |
| 0       | 0       | 0       | 0       |

During the value iteration:

| -0.01   | -0.01   | 0.782   | ✅       |
| -0.01   | ▉       | -0.01   | 💀       |
| -0.01   | -0.01   | -0.01   | -0.01   |

| -0.01   | 0.607   | 0.858   | ✅       |
| -0.01   | ▉       | 0.509   | 💀       |
| -0.01   | -0.01   | -0.01   | -0.01   |

| 0.467   | 0.790   | 0.917   | ✅       |
| -0.02   | ▉       | 0.621   | 💀       |
| -0.02   | -0.02   | 0.389   | -0.02   |

| 0.659   | 0.873   | 0.934   | ✅       |
| 0.354   | ▉       | 0.679   | 💀       |
| -0.03   | 0.292   | 0.476   | 0.196   |

| 0.781   | 0.902   | 0.941   | ✅       |
| 0.582   | ▉       | 0.698   | 💀       |
| 0.295   | 0.425   | 0.576   | 0.287   |

| 0.840   | 0.914   | 0.944   | ✅       |
| 0.724   | ▉       | 0.705   | 💀       |
| 0.522   | 0.530   | 0.613   | 0.375   |

| 0.869   | 0.919   | 0.945   | ✅       |
| 0.798   | ▉       |